https://norahsakal.com/blog/fine-tune-gpt3-model

In [7]:
import os
import pandas as pd
from typing import Dict
import openai, json, chromadb
from chromadb.utils import embedding_functions
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm

In [8]:
openai.api_key = open("key.txt", "r").read()
os.environ["OPENAI_API_KEY"] = openai.api_key

##  ***Data preparation***

In [9]:
#   Read data from text
with open("chatbot/data/start3D_DB.txt", "r") as file:
    text = file.read()
len(text.split())

319444

### ***Text chunking***

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
)

#   Remove repeat chunk
chunks = list(set(text_splitter.split_text(text)))

In [11]:
embed_func = embedding_functions.OpenAIEmbeddingFunction(
                            api_key=os.environ["OPENAI_API_KEY"],
                            model_name="text-embedding-ada-002"
                )
        
# Create a PersistentClient and get or create the collection
client = chromadb.PersistentClient("./chromadb__")
collection = client.get_or_create_collection(name="start3D", 
                                             embedding_function=embed_func, 
                                             metadata={"hnsw:space": "cosine"})

# Add documents to the collection
collection.add(
    documents=chunks,   #   [paragraph1, paragraph2]
    metadatas=[{"source": "notion"}]*len(chunks),
    ids = [str(i) for i in range(len(chunks))]
)

In [12]:
collection

Collection(name=start3D)

In [13]:
def text_embedding(text):
    response = openai.Embedding.create(model="text-embedding-ada-002", input=text)
    return response["data"][0]["embedding"]

### ***Multiple requests***

In [ ]:
#   Multiple queries
queries_text = ["Đảng Cộng sản Việt Nam được thành lập vào ngày nào?",
                "Ai đã nhận được sự đồng tình ủng hộ của nhân dân các nước yêu chuộng hòa bình trong cuộc kháng chiến giành độc lập dân tộc?"]
queries = []
for query in queries_text:
    queries.append(text_embedding(query))

queries

In [12]:
results = collection.query(
    query_embeddings=queries,
    n_results=1
)
results

{'ids': [['496'], ['185']],
 'distances': [[0.10070765018463135], [0.0912785530090332]],
 'metadatas': [[{'source': 'notion'}], [{'source': 'notion'}]],
 'embeddings': None,
 'documents': [['Ngày 3/2/1930 Đảng Cộng sản Việt Nam được thành lập. Đây là sự kiện lịch sử có ý nghĩa đặc biệt quan trọng của cách mạng Việt Nam. Ban Chấp hành Trung ương Đảng thời kỳ đầu hoạt động chủ yếu tại địa bàn Thành phố đã bí mật xây dựng cơ sở, truyền bá sách báo tiến bộ, tập hợp những công nhân ưu tú trong các nhà máy, hãng xưởng làm hạt nhân vận động, tổ chức các cuộc đấu tranh thu hút các tầng lớp đồng bào. Từ tháng 3 năm 1930 tại Thành phố cũng diễn ra việc hợp nhất thành lập các Chi bộ cộng sản, thành lập Ban lâm thời chấp ủy của Đảng bộ Sài Gòn do đồng chí Nguyễn Văn Lợi (Hữu Dũng) làm Bí thư. Đảng bộ Thành phố có khoảng 20 Chi bộ với 130 Đảng viên. Phòng trưng bày đấu tranh cách mạng giai đoạn 1930 – 1954 là phòng số 5 (lầu 1) trong hệ thống trưng bày của Bảo tàng Thành phố, tổng cộng 301 hiện vật

### ***Single request***

In [48]:
embed_vec = text_embedding("Đảng Cộng sản Việt Nam được thành lập vào ngày nào?")

results = collection.query(
    query_embeddings=embed_vec,
    include=["documents", "metadatas"],
    n_results=1
)
result_text = "\n".join(str(item) for item in results['documents'][0])

In [52]:
results

{'ids': [['319']],
 'distances': None,
 'metadatas': [[{'source': 'notion'}]],
 'embeddings': None,
 'documents': [['Ngày 3/2/1930 Đảng Cộng sản Việt Nam được thành lập. Đây là sự kiện lịch sử có ý nghĩa đặc biệt quan trọng của cách mạng Việt Nam. Ban Chấp hành Trung ương Đảng thời kỳ đầu hoạt động chủ yếu tại địa bàn Thành phố đã bí mật xây dựng cơ sở, truyền bá sách báo tiến bộ, tập hợp những công nhân ưu tú trong các nhà máy, hãng xưởng làm hạt nhân vận động, tổ chức các cuộc đấu tranh thu hút các tầng lớp đồng bào. Từ tháng 3 năm 1930 tại Thành phố cũng diễn ra việc hợp nhất thành lập các Chi bộ cộng sản, thành lập Ban lâm thời chấp ủy của Đảng bộ Sài Gòn do đồng chí Nguyễn Văn Lợi (Hữu Dũng) làm Bí thư. Đảng bộ Thành phố có khoảng 20 Chi bộ với 130 Đảng viên. Phòng trưng bày đấu tranh cách mạng giai đoạn 1930 – 1954 là phòng số 5 (lầu 1) trong hệ thống trưng bày của Bảo tàng Thành phố, tổng cộng 301 hiện vật bao gồm: hình ảnh, tài liệu, báo chí và các loại vũ khí… tất cả là những 

In [50]:
locations = results["metadatas"][0]
answers = results["documents"][0]

final_text = []
old_location = ''
for location, answer in zip(locations, answers):
    if location["source"] != old_location:
        final_text.append("Dưới đây là thông tin về " + location["source"] + ":")
        old_location = location["source"]
    final_text.append(answer)
final_text = "\n".join(final_text)

In [ ]:
{'ids': [['0']],
 'distances': None,
 'metadatas': [[{'source': 'Tòa nhà Landmark81'}]],
 'embeddings': None,
 'documents': [['Landmark 81 cao ....']]}

### ***Update Database***

In [13]:
curr_id = len(chunks)
curr_id

522

In [16]:
new_chunks = ["Ngày 3/2/1930 Đảng Cộng sản Việt Nam được thành lập. Đây là sự kiện lịch sử có ý nghĩa đặc biệt quan trọng của cách mạng Việt Nam. Ban Chấp hành Trung ương Đảng",
              "Năm 1930, Đảng Cộng sản Đông Dương thành lập, đã đề ra cương lĩnh Cách Mạng đúng đắn. Trong 15 năm lãnh đạo nhân dân Việt Nam thực hiện cuộc Cách Mạng Tháng 8 năm 1945, đánh đổ",
              "Tầng 2 & 1: Mỹ thuật hiện đại, Phòng trải nghiệm mỹ thuật, Quầy lưu niệm, Vườn tượng, Phòng tranh. Nguồn: Bảo tàng Mỹ thuật TP.HCM PHÒNG TRƯNG BÀY BẢO VẬT QUỐC GIA",
              "Tòa nhà 1: Đây là nơi trưng bày các tác phẩm mỹ thuật hiện đại của các họa sĩ nổi tiếng trường Gia Định, Mỹ thuật Đông Dương và mỹ thuật hiện đại từ trước năm 1975"]

In [17]:
collection.upsert(
    ids=[str(curr_id+i) for i in range(len(new_chunks))],
    documents=new_chunks,
)